In [38]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score
from bs4 import BeautifulSoup
import os

import warnings
warnings.filterwarnings("ignore")

In [39]:
os.chdir('C:/Users/jean_/dev/github/EDA/Documents/database')
diretorio_atual = os.getcwd()

df_imdb = pd.DataFrame(pd.read_csv('IMDB_eda.csv', sep = ',', decimal = '.'))

In [40]:
df_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [41]:
df_imdb.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [42]:
df_imdb['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [43]:
df_imdb['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [44]:
def retirando_tag(texto):
    soup = BeautifulSoup(texto, "html.parser")
    return soup.get_text()

def remover_caracter_especial(texto):
    remover = ''
    for i in texto:
        if i.isalnum():
            remover = remover + i
        else:
            remover = remover + ' '
    
    return remover

def tudo_minuscula(texto):
    return texto.lower()

def limpando_todo_texto(texto):
    texto = retirando_tag(texto)
    texto = remover_caracter_especial(texto)
    texto = tudo_minuscula(texto)

    return texto

In [45]:
df_imdb['review'] = df_imdb['review'].apply(limpando_todo_texto)

In [46]:
nltk.download('stopwords')
nltk.download('punkt')

def removendo_stopword(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)

    return [w for w in words if w not in stop_words]

def stemming_texto(texto):
    ss = SnowballStemmer('english')

    return ' '.join([ss.stem(w) for w in texto])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jean_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jean_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [47]:
df_imdb['review'] = df_imdb['review'].apply(removendo_stopword)

In [48]:
df_imdb['review'][0]

['one',
 'reviewers',
 'mentioned',
 'watching',
 '1',
 'oz',
 'episode',
 'hooked',
 'right',
 'exactly',
 'happened',
 'first',
 'thing',
 'struck',
 'oz',
 'brutality',
 'unflinching',
 'scenes',
 'violence',
 'set',
 'right',
 'word',
 'go',
 'trust',
 'show',
 'faint',
 'hearted',
 'timid',
 'show',
 'pulls',
 'punches',
 'regards',
 'drugs',
 'sex',
 'violence',
 'hardcore',
 'classic',
 'use',
 'word',
 'called',
 'oz',
 'nickname',
 'given',
 'oswald',
 'maximum',
 'security',
 'state',
 'penitentary',
 'focuses',
 'mainly',
 'emerald',
 'city',
 'experimental',
 'section',
 'prison',
 'cells',
 'glass',
 'fronts',
 'face',
 'inwards',
 'privacy',
 'high',
 'agenda',
 'em',
 'city',
 'home',
 'many',
 'aryans',
 'muslims',
 'gangstas',
 'latinos',
 'christians',
 'italians',
 'irish',
 'scuffles',
 'death',
 'stares',
 'dodgy',
 'dealings',
 'shady',
 'agreements',
 'never',
 'far',
 'away',
 'would',
 'say',
 'main',
 'appeal',
 'show',
 'due',
 'fact',
 'goes',
 'shows',
 'da

In [49]:
def stemming_texto(texto):
    ss = SnowballStemmer('english')

    return ' '.join([ss.stem(w) for w in texto])

In [50]:
df_imdb['review'] = df_imdb['review'].apply(stemming_texto)

In [51]:
df_imdb['review'][0]

'one review mention watch 1 oz episod hook right exact happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focus main emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch darker side'

In [52]:
x = np.array(df_imdb['review'])
y = np.array(df_imdb['sentiment'])
cv = CountVectorizer(max_features = 1000)
x = cv.fit_transform(df_imdb['review']).toarray()

In [53]:
print('x.shape = ', x.shape)
print('y.shape = ', y.shape)

x.shape =  (50000, 1000)
y.shape =  (50000,)


In [54]:
print(x)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [55]:
trainx, testx, trainy, testy = train_test_split(x, y, test_size = 0.2, random_state = 9)
print('Train shapes: x = {}, y = {}'.format(trainx.shape, trainy.shape))
print('Test shapes: x = {}, y = {}'.format(testx.shape, testy.shape))

Train shapes: x = (40000, 1000), y = (40000,)
Test shapes: x = (10000, 1000), y = (10000,)


In [56]:
gnb, mnb, bnb = GaussianNB(), MultinomialNB(alpha = 1.0, fit_prior = True), BernoulliNB(alpha = 1.0, fit_prior = True)

gnb.fit(trainx, trainy)
mnb.fit(trainx, trainy)
bnb.fit(trainx, trainy)

BernoulliNB()

In [57]:
ypg = gnb.predict(testx)
ypm = mnb.predict(testx)
ypb = bnb.predict(testx)

In [58]:
print('Gaussian: ', accuracy_score(testy, ypg))
print('Multinomial: ', accuracy_score(testy, ypm))
print('Bernoulli: ', accuracy_score(testy, ypb))

Gaussian:  0.7843
Multinomial:  0.8309
Bernoulli:  0.8387


In [59]:
review = '''
But the movie has plot as well. 
It has characters that I cared about.
From Keanu Reeves' excellent portrayal of Neo, the man trying to come to grips with his own identity, to Lawrence Fishburne's mysterious Morpheus, and even the creepy Agents, everyone does a stellar job of making their characters more than just the usual action "hero that kicks butt" and "cannon fodder" roles.
I cared about each and every one of the heroes, and hated the villains with a passion.
It has a plot, and it has a meaning...and lo and behold, a plot does help the fight scenes!
Just try it, if you haven't seen the movie before.  Watch one of the fight scenes.
Then watch the whole movie. 
There's a big difference in the feeling and excitement of the scenes- sure, they're great as standalones, but the whole thing put together is an experience unlike just about everything else that's come to the theaters.
Think about it next time you're watching one of the more brainless action flicks...think how much better it COULD be.
'''

passo_1 = limpando_todo_texto(review)
passo_2 = removendo_stopword(passo_1)
passo_3 = stemming_texto(passo_2)

bow, palavras = [], word_tokenize(passo_3)
for palavra in palavras:
    bow.append(palavras.count(palavra))

dic_palavra = cv.vocabulary_
inp = []
for i in dic_palavra:
    inp.append(passo_3.count(i[0]))
y_pred = bnb.predict(np.array(inp).reshape(1, 1000))

In [60]:
print(y_pred)

['positive']


In [64]:
review = '''
I slept for 37 hours, woke up and then I drank 7 red bulls, 16 Five hour energys and 4 lines of cocaine and I watched this movie standing in the shower on cold. I still feel asleep during this film.
'''

passo_1 = limpando_todo_texto(review)
passo_2 = removendo_stopword(passo_1)
passo_3 = stemming_texto(passo_2)

bow, palavras = [], word_tokenize(passo_3)
for palavra in palavras:
    bow.append(palavras.count(palavra))

dic_palavra = cv.vocabulary_
inp = []
for i in dic_palavra:
    inp.append(passo_3.count(i[0]))
y_pred = bnb.predict(np.array(inp).reshape(1, 1000))

print(y_pred)

['negative']
